1. 机器学习的基本假设是：      
   A. 在训练和测试数据中存在相同的模式。  
   B. 训练数据和测试数据数量相同。   
   C. 测试数据是训练数据的一部分。   
   D. 训练数据的量足够多，可以训练出好的模型。  

答：  
A. 正确。训练数据和测试数据具有假设空间的一致性和数据分布的一致性时（独立同分布），才能从训练数据上学到的模式用测试数据进行测试并优化。不然模型在训练数据上学到的模式和测试数据差异很大就是过拟合了。    
B. 错误。一般测试集会比较小。测试数据是用来评估模型的泛化能力的，训练数据一般比较大模型才更能学到模式表达数据特征。    
C. 错误。测试数据必须是模型没有见过的，不然模型都已经学习过这些数据了，就不能称之为测试。（考试作弊）。  
D. 错误。训练数据并不是量越大越好，好的模型取决于设计的模式是否拟合数据的特征。训练数据过大会导致计算资源消耗大，学到不恰当的特征等问题。  

---

2. 下列说法正确的是：   
   A. 为了防止过拟合，正则化系数越大越好。   
   B. 为了防止过拟合，应当选择尽可能简单的模型。   
   C. 交叉验证方法中虽然所有数据都被用到，但是不存在信息泄露。   
   D. 正则化系数$\lambda$可以通过令损失函数对$\lambda$的梯度为零计算出来。  

答：  
A. 错误。正则化系数一般设置为比较小的数。系数过大的话惩罚严重会导致模型无法拟合数据，欠拟合。  
B. 正确。简单的模型更容易泛化到新的数据上，但是模型也不能过于简单。   
C. 正确。先划分数据集再进行训练确保了信息保密，同时划分数据集前一般还会对数据进行随机打乱处理。  
D. 错误。正则化系数是手动选择的超参数，需要通过模型的效果人工调试。  

---

3. 假如数据中不存在噪声，过拟合现象是否会消失？

答：不会。过拟合不仅仅是针对噪声，模型需要学习的模式是能概括到整体数据的，如果训练数据比较复杂，也会过度拟合数据导致泛化能力较差。训练数据的某些细节是需要忽视的，不能完全拟合数据。解决过拟合问题可以采样正则化和特征选择等。

---

4. 机器学习模型是否可以预测毫无规律的真随机数？试从统计规律的角度解释原因。

答：不能。机器学习模型其实就是学习训练数据的统计规律来建立预测模型的。而真随机数本就是不可预测的随机数值，每个数字都是独立的，是不可能有模式能够描述的。

---

5. 除了学习率$\eta$，第 4 章的线性回归模型中还有哪些超参数？数据集$N$大小是超参数吗？

答：超参数是机器学习模型中不通过训练数据得到而是训练前设置的参数，超参数需要人为通过交叉验证判断进行设置或者通过启发式算法搜索来找到合适的超参数。  
第4章的线性回归模型中除了学习率，梯度下降的迭代次数、批量大小以及收敛停止的条件也是超参数。数据集大小通常不是超参数，这是一个外部给定的属性，取决于数据集的大小。

---

6. 在实践中，如果模型在测试集上的效果不好，如何判断模型是欠拟合还是过拟合？

答：我的经验而言，会把模型的训练误差和测试误差都画出来，如果明显看到训练误差小，测试误差大，就是过拟合了；如果发现训练误差和测试误差都不是很好，就是可能欠拟合了。还可以看看模型训练出来的参数，如果模型参数很复杂很大，参数比较多，就是可能过拟合了，如果模型复杂度很低，参数很少，就可能欠拟合了。

---

7. 将第4章中的训练集-验证集划分改为交叉验证，选出最好的模型在测试集上测试

In [57]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


# 从源文件加载数据，并输出查看数据的各项特征
lines = np.loadtxt('D:/我的学习/Machine_Learning/动手学机器学习/Hands-on-ML/第4章 线性回归/USA_Housing.csv', delimiter=',', dtype='str')
header = lines[0]
lines = lines[1:].astype(float)
print('数据特征：', ', '.join(header[:-1]))
print('数据标签：', header[-1])
print('数据总条数：', len(lines))


# 数据归一化
scaler = StandardScaler()
scaler.fit(lines) # 使用所有数据计算均值和方差
lines_scaled = scaler.transform(lines)

# 划分输入和标签
x, y = lines_scaled[:, :-1], lines_scaled[:, -1]

# 定义线性回归模型
model = LinearRegression()

# 使用交叉验证来选择最佳模型
cv_scores = cross_val_score(model, x, y, cv=5, scoring='neg_mean_squared_error')
mse_scores = -cv_scores  # 取负值得到均方误差

print("各模型的均方误差：", mse_scores)

best_mse = mse_scores.max()  # 选择均方误差最小的模型
best_model_index = mse_scores.argmax()  # 最佳模型对应的索引

print("最佳模型的均方误差：", best_mse)

# 在测试集上评估最佳模型
train_size = len(lines) * 4 // 5  # 使用80%的数据作为训练集
test_size = len(lines) - train_size  # 剩下的数据作为测试集
train, test = lines_scaled[:train_size], lines_scaled[train_size:]
x_train, y_train = train[:, :-1], train[:, -1]
x_test, y_test = test[:, :-1], test[:, -1]

best_model = model.fit(x_train, y_train)  # 在训练集上训练最佳模型
y_pred = best_model.predict(x_test)  # 在测试集上预测
mse_test = mean_squared_error(y_test, y_pred)  # 计算在测试集上的均方误差
print("最佳模型在测试集上的均方误差：", mse_test)

# 输出最佳模型的参数
print("最佳模型的参数：")
print("系数（斜率）：", best_model.coef_)
print("截距：", best_model.intercept_)


数据特征： Avg. Area Income, Avg. Area House Age, Avg. Area Number of Rooms, Avg. Area Number of Bedrooms, Area Population
数据标签： Price
数据总条数： 5000
各模型的均方误差： [0.08091231 0.08605404 0.07910359 0.0816279  0.08354542]
最佳模型的均方误差： 0.0860540387450174
最佳模型在测试集上的均方误差： 0.08354542259852639
最佳模型的参数：
系数（斜率）： [0.65361133 0.46378742 0.34520317 0.00353179 0.42506886]
截距： 0.002706822695746307
